In [ ]:
import requests 
import json
import pandas as pd

#to see completed rebalancing orders:
# https://ftx.com/api/lt/ETHBULL/major_rebalances

In [ ]:
# any token will rebalance if an intraday move causes its leverage to be 33% higher than its target.  
# So if markets move down enough that BULL token is 4x leveraged it will rebalance.  
# This corresponds to market moves of roughly 11.15% for BULL tokens, 6.7% for BEAR tokens, and 30% for HEDGE tokens.


# would require a 33% market move to liquidate a 3x leveraged token, 
# but the token will generally rebalance within a 6-12% market move, reducing its risk and returning to 3x leveraged.



In [ ]:
pd.options.display.float_format = '${:,.2f}'.format

dframe = pd.DataFrame()
lt = json.loads(requests.get('https://ftx.com/api/lt/tokens').text)
lt['result']
componentadd = []
for e in lt['result']:
    componentadd.append(e['underlying'])
underlyingset = set(componentadd)
underlying = list(underlyingset)
underlying.sort()


for e in underlying:
    undstatreq = json.loads(requests.get(f'https://ftx.com/api/futures/{e}/stats').text)
    underlyinglt = []
    #print(e)
    for r in lt['result']:
        if r['underlying'] == e:
            underlyinglt.append(r)
        
        rebalamt = []
        for t in underlyinglt:
           
            
            t['exposure_current'] = t['outstanding'] * t['underlyingMark'] * t['positionPerShare']
            t['exposure_target'] = t['totalCollateral'] * t['leverage']
            t['leverage_current'] = t['exposure_current'] / t['totalCollateral']
            t['rebal'] = t['exposure_target'] - t['exposure_current']
            
            rebalamt.append(t['rebal'])
            
            


        
    dframe = dframe.append({'und':e,'rebal':sum(rebalamt),'px':t['underlyingMark'],'volume':(undstatreq['result']['volume']*t['underlyingMark'])},ignore_index=True)



dframe = dframe.reindex(dframe.rebal.abs().sort_values(ascending=False).index)
dframe = dframe[['und', 'rebal', 'px', 'volume']]
dframe
